<a href="https://colab.research.google.com/github/frankayra/Conv1D/blob/main/Conv1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# importacion 'os'

In [1]:
import os

# Montando Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importando y descargando GTZAN

### No pincha 👇🏻

In [ ]:
import tensorflow_datasets as tfds
dataset = tfds.load('gtzan', split='train')

############## NO PINCHA ##############

In [ ]:
!pip install mirdata

import mirdata

# INICIALIZANDO EL DATASET (GTZAN)
gtzan = mirdata.initialize('gtzan_genre', data_home=path)

# DESCARGARLO (SOLO UNA VEZ)
gtzan.download(partial_download=['all']) # Comentar luego de descargar el dataset

############## NO PINCHA ##############

### ESTO ES LO QUE PINCHA 👇🏻

In [ ]:
!pip install kaggle
api_token = {"username":"franciscoayracacceres","key":"0faed9196b5d77ac39f381ebcbcea42e"}

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle

In [8]:
!cp kaggle.json ~/.kaggle/

In [9]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

In [28]:
!kaggle config set -n path -v drive/MyDrive/ML_DATABASE/

- path is now set to: drive/MyDrive/ML_DATABASE/


In [29]:
!kaggle datasets download --unzip andradaolteanu/gtzan-dataset-music-genre-classification --force

100% 1.21G/1.21G [00:30<00:00, 45.5MB/s]
100% 1.21G/1.21G [00:30<00:00, 42.5MB/s]


In [3]:
os.listdir("drive/MyDrive/ML_DATABASE")

['gtzan-features.zip',
 'gtzan-features uploaded',
 'GTZAN',
 '.kaggle',
 '.ipynb_checkpoints',
 'datasets']

---
---

# Conv1D

### Importaciones

In [9]:
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten
from keras.layers import BatchNormalization, Activation
from keras.utils import to_categorical

### Asegurando el uso de gpu para la creacion, sosten y uso de la red convolucional

In [10]:
device_name = tf.test.gpu_device_name()

assert device_name == '/device:GPU:0', "mira la nota a la derecha para activar el uso de GPU"

print('se encontro una gpu')

se encontro una gpu


In [13]:
path = "/content/drive/My Drive/ML_DATABASE/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/Data/genres_original"
assert os.listdir(path)

### Creacion del Modelo

In [14]:
# CARGAR UN ARCHIVO DE AUDIO
def load_audio_file(file_path):
    input_length = 660000  # This is 15 seconds with 44100 sample rate
    data = librosa.core.load(file_path, sr=22050)[0]  # We use librosa to load audio file with sample rate 22050
    if len(data) > input_length:
        data = data[:input_length]
    else:
        data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
    return data


In [15]:
# CARGAR EL CONJUNTO DE DATOS
def load_dataset(path):
    genres = os.listdir(path)
    X, y = [], []
    for genre in genres:
        genre_path = os.path.join(path, genre)
        for file_name in os.listdir(genre_path):
            file_path = os.path.join(genre_path, file_name)
            try:
              data = load_audio_file(file_path)
              print(f"audio {file_name} cargado")
            except:
              print(f"error con {file_name}")
              continue
            X.append(data)
            y.append(genre)
    return np.array(X), np.array(y)



In [ ]:
X, y = load_dataset(path)

with tf.device('/device:GPU:0'):

  # ENCODE LABELS
  encoder = LabelEncoder()
  y = encoder.fit_transform(y)
  y = to_categorical(y, num_classes=10)  # Assuming 10 genres

  # PARTICIONAR LOS DATOS
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

  # REMODELAR LOS DATOS
  X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
  X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
  X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

  # MODELO DE CNN 1D BASADO EN LA ARQUITECTURA ResNet
  model = Sequential()

  # PRIMERA CAPA DE CONVOLUCION
  model.add(Conv1D(64, kernel_size=3, strides=1, padding="same", input_shape=(X_train.shape[1], 1)))
  model.add(BatchNormalization())
  model.add(Activation("relu"))

  # BLOQUES RESIDUALES
  for _ in range(3):
      model.add(Conv1D(64, kernel_size=3, strides=1, padding="same"))
      model.add(BatchNormalization())
      model.add(Activation("relu"))
      model.add(Conv1D(64, kernel_size=3, strides=1, padding="same"))
      model.add(BatchNormalization())
      model.add(Activation("relu"))

  # CAPA DE POOLING
  model.add(MaxPooling1D(pool_size=2, strides=2))

  # CAPA CONECTADA COMPLETAMENTE
  model.add(Flatten())
  model.add(Dense(256, activation="relu"))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation="softmax"))  # Assuming 10 genres

  # COMPILACION DEL MODELO
  model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

  # ENTRENAMIENTO
  model.fit(X_train, y_train, batch_size=80, epochs=100, validation_data=(X_val, y_val))


audio blues.00000.wav cargado
audio blues.00001.wav cargado
audio blues.00002.wav cargado
audio blues.00003.wav cargado
audio blues.00004.wav cargado
audio blues.00005.wav cargado
audio blues.00006.wav cargado
audio blues.00007.wav cargado
audio blues.00008.wav cargado
audio blues.00009.wav cargado
audio blues.00010.wav cargado
audio blues.00011.wav cargado
audio blues.00012.wav cargado
audio blues.00013.wav cargado
audio blues.00014.wav cargado
audio blues.00015.wav cargado
audio blues.00016.wav cargado
audio blues.00017.wav cargado
audio blues.00018.wav cargado
audio blues.00019.wav cargado
audio blues.00020.wav cargado
audio blues.00021.wav cargado
audio blues.00022.wav cargado
audio blues.00023.wav cargado
audio blues.00024.wav cargado
audio blues.00025.wav cargado
audio blues.00026.wav cargado
audio blues.00027.wav cargado
audio blues.00028.wav cargado
audio blues.00029.wav cargado
audio blues.00030.wav cargado
audio blues.00031.wav cargado
audio blues.00032.wav cargado
audio blue

<ipython-input-14-a661506bab05>:4: UserWarning: PySoundFile failed. Trying audioread instead.
  data = librosa.core.load(file_path, sr=22050)[0]  # We use librosa to load audio file with sample rate 22050
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


audio metal.00001.wav cargado
audio metal.00002.wav cargado
audio metal.00003.wav cargado
audio metal.00004.wav cargado
audio metal.00005.wav cargado
audio metal.00006.wav cargado
audio metal.00007.wav cargado
audio metal.00008.wav cargado
audio metal.00009.wav cargado
audio metal.00010.wav cargado
audio metal.00011.wav cargado
audio metal.00012.wav cargado
audio metal.00013.wav cargado
audio metal.00014.wav cargado
audio metal.00015.wav cargado
audio metal.00016.wav cargado
audio metal.00017.wav cargado
audio metal.00018.wav cargado
audio metal.00019.wav cargado
audio metal.00020.wav cargado
audio metal.00021.wav cargado
audio metal.00022.wav cargado
audio metal.00023.wav cargado
audio metal.00024.wav cargado
audio metal.00025.wav cargado
audio metal.00026.wav cargado
audio metal.00027.wav cargado
audio metal.00028.wav cargado
audio metal.00029.wav cargado
audio metal.00030.wav cargado
audio metal.00031.wav cargado
audio metal.00032.wav cargado
audio metal.00033.wav cargado
audio meta